In [1]:
import time
import numpy as np
import cv2
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy import signal
import os, errno
import glob

In [3]:
# load centroid file wih distance and centroid data

df = pd.read_csv('./dataFolders/Output/KinematicAnalysis_Final/CentroidData.csv')
df.head()

,Unnamed: 0,name,visit,flower_r,flower_x,flower_y,x,y,x_pixel,y_pixel
0,0,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
1,1,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
2,2,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
3,3,L0.1_c-3_m10,last,74,310,233,0.810811,2.297297,24.0,68.0
4,4,L0.1_c-3_m10,last,74,310,233,1.013514,2.770270,30.0,82.0


In [9]:
df_use = df.copy()
df_use = df_use.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 
                                'distance', 'distanceincms',
                                'cent_x', 'cent_y'
                                ])

In [5]:
df_use = df.copy()
df_use = df_use.drop(columns = ['Unnamed: 0'])

In [6]:
df_use.head()

,name,visit,flower_r,flower_x,flower_y,x,y,x_pixel,y_pixel
0,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
1,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
2,L0.1_c-3_m10,last,74,310,233,NaN,NaN,NaN,NaN
3,L0.1_c-3_m10,last,74,310,233,0.810811,2.297297,24.0,68.0
4,L0.1_c-3_m10,last,74,310,233,1.013514,2.770270,30.0,82.0


## smoothen data using a low pass filter

In [7]:
from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [8]:
sf = 20 # set smoothening factor for cubic spline interpolation

In [9]:
def filterTimeseries(test, lowF):
    x_interpl = test.x_pixel.interpolate(method = 'cubic')
    y_interpl = test.y_pixel.interpolate(method = 'cubic')
    test_interpl = pd.DataFrame({'x' : x_interpl,
                                'y' : y_interpl})

    # trim the NaN edges 
    first_idx = test_interpl.first_valid_index()
    last_idx = test_interpl.last_valid_index()

    test_filt = test_interpl.loc[first_idx: last_idx, :].copy()

    filt_x = butter_lowpass_filter(test_filt.x.values, lowF, 100, order = 5)
    filt_y = butter_lowpass_filter(test_filt.y.values, lowF, 100, order = 5)

    test_filt.loc[:,'x'] = filt_x
    test_filt.loc[:,'y'] = filt_y

    # add the trimmed edges back to filtered timeseries
    ll =[]
    missing_ind = first_idx - test.index[0]
    if missing_ind != 0:
        for n in np.arange(missing_ind):
            ll.append(first_idx-n-1)

    missing_ind = test.index[-1] - last_idx
    if missing_ind != 0:
        for n in np.arange(missing_ind):
            ll.append(last_idx+n+1)

    for l in ll:
        test_filt.loc[l, :] = np.nan
    test_filt = test_filt.sort_index()

    # reset original values back to NaN
    w_x = np.isnan(test.x_pixel)
    w_y = np.isnan(test.y_pixel)

    test_filt.loc[w_x, 'x'] = np.nan
    test_filt.loc[w_y, 'y'] = np.nan
    return(test_filt)

In [10]:
#create a pd with all smoothened x and y centroids in pixel space

mothname = []
visit = []
flower_r = []
flower_x = []
flower_y = []
centroid_x = []
centroid_y = []
p_x = []
p_y = []
f_x = []
f_y = []

lowF = 18
for name, sub_df in df_use.groupby('name'):
    filtered = filterTimeseries(sub_df, 18)
    filtered.x  = filtered.x
    filtered.y  = filtered.y
    
    r = sub_df.flower_r
    circ_x = sub_df.flower_x
    circ_y = sub_df.flower_y
    n = sub_df.name
    v = sub_df.visit
    x = sub_df.x
    y = sub_df.y
    pixel_x = sub_df.x_pixel
    pixel_y = sub_df.y_pixel
    
    mothname.extend(n)
    visit.extend(v)
    flower_r.extend(r)
    flower_x.extend(circ_x)
    flower_y.extend(circ_y)
    centroid_x.extend(x)
    centroid_y.extend(y)
    p_x.extend(pixel_x)
    p_y.extend(pixel_y)
    f_x.extend(filtered.x)
    f_y.extend(filtered.y)
    
#     print("%s: filtered track is %d long and rest of %d long" %(name, len(filtered.x), len(n)))

In [11]:
uber_df = pd.DataFrame({'name' : mothname,
                        'visit' : visit,
                       'flower_r': flower_r,
                       'flower_x': flower_x,
                       'flower_y': flower_y,
                        'x':centroid_x,
                        'y':centroid_y,
                        'x_pixel': p_x,
                        'y_pixel': p_y,
                        'x_filtered':f_x,
                        'y_filtered':f_y
                       })
uber_df.to_csv('./dataFolders/Output/KinematicAnalysis_Final/LowPassFilteredCentroidInPixelsSpace.csv')